In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# 1. DATEN LADEN UND VORBEREITEN
# =============================================================================
def load_and_prepare_data():
    """Lädt die Daten und bereitet sie für das Modelling vor."""
    print("Lade Daten...")
    
    # Daten laden
    df = pd.read_csv('/workspaces/bakery_sales_prediction/2_BaselineModel/data/data_clean.csv')
    sample_sub = pd.read_csv('/workspaces/bakery_sales_prediction/2_BaselineModel/datasets/sample_submission.csv')
    
    # Datum konvertieren
    df['Datum'] = pd.to_datetime(df['Datum'])
    
    # Zusätzliche Zeit-Features erstellen
    df['Tag'] = df['Datum'].dt.day
    df['Monat'] = df['Datum'].dt.month
    df['Jahr'] = df['Datum'].dt.year
    df['Wochentag'] = df['Datum'].dt.dayofweek  # 0=Montag, 6=Sonntag
    
    # Wetter-Features erweitern
    # Wetter-Cluster erstellen
    df['wetter_cluster'] = df['Wettercode'].apply(create_weather_clusters)
    
    # One-Hot-Encoding für Wetter-Cluster
    wetter_dummies = pd.get_dummies(df['wetter_cluster'], prefix='Wetter')
    df = pd.concat([df, wetter_dummies], axis=1)
    
    # Temperaturkategorien
    df['Temperatur_Kategorie'] = pd.cut(df['Temperatur'], 
                                      bins=[-float('inf'), 10, 20, 25, float('inf')],
                                      labels=['kalt', 'mild', 'warm', 'heiss'])
    
    # One-Hot-Encoding für Temperaturkategorien
    temp_dummies = pd.get_dummies(df['Temperatur_Kategorie'], prefix='Temp')
    df = pd.concat([df, temp_dummies], axis=1)
    
    # Temperaturänderung zum Vortag
    df['Temperatur_Delta'] = df.groupby(['Jahr', 'Monat'])['Temperatur'].diff()
    
    # Zyklische Zeitfeatures
    df['tag_sin'] = np.sin(2 * np.pi * df['Tag']/31)
    df['tag_cos'] = np.cos(2 * np.pi * df['Tag']/31)
    df['monat_sin'] = np.sin(2 * np.pi * df['Monat']/12)
    df['monat_cos'] = np.cos(2 * np.pi * df['Monat']/12)
    
    # Jahreszeiten (meteorologisch)
    df['Jahreszeit'] = df['Monat'].map({
        12:0, 1:0, 2:0,  # Winter
        3:1, 4:1, 5:1,   # Frühling
        6:2, 7:2, 8:2,   # Sommer
        9:3, 10:3, 11:3  # Herbst
    })
    
    print(f"Daten geladen: {len(df)} Datensätze von {df['Datum'].min()} bis {df['Datum'].max()}")
    print(f"Sample Submission: {len(sample_sub)} Vorhersagen benötigt")
    
    return df, sample_sub

In [ ]:
# 2. DATEN AUFTEILEN
# =============================================================================

def split_data(df):
    """Teilt die Daten in Train/Validation/Test auf."""
    print("\nTeile Daten auf...")
    
    # Zeitbasierte Aufteilung
    train = df[df['Datum'] < '2017-08-01'].copy()
    validation = df[(df['Datum'] >= '2017-08-01') & (df['Datum'] < '2018-08-01')].copy()
    
    print(f"Training: {len(train)} Datensätze ({train['Datum'].min()} - {train['Datum'].max()})")
    print(f"Validation: {len(validation)} Datensätze ({validation['Datum'].min()} - {validation['Datum'].max()})")
    
    return train, validation

In [ ]:
# 3. FEATURES DEFINIEREN
# =============================================================================

def get_features(df=None):
    """Definiert die Features für die Regression."""
    # Basis-Wetter-Features
    weather_features = [
        'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode',
        'Wettercode_fehlt', 'Temperatur_Delta'
    ]
    
    # Temperatur-Kategorie Features
    temp_features = [col for col in df.columns if col.startswith('Temp_')] if df is not None else [
        'Temp_kalt', 'Temp_mild', 'Temp_warm', 'Temp_heiss'
    ]
    
    # Wetter-Cluster Features
    weather_cluster_features = [col for col in df.columns if col.startswith('Wetter_')] if df is not None else [
        'Wetter_Bewölkung', 'Wetter_Sichtminderung', 'Wetter_Nebel',
        'Wetter_Niederschlag', 'Wetter_Schauer', 'Wetter_Gewitter',
        'Wetter_Extremwetter', 'Wetter_Fehlt'
    ]
    
    # Feiertags-Features
    holiday_features = [
        'ist_feiertag', 'feiertag_vortag', 'feiertag_folgetag',
        'KielerWoche'
    ]
    
    # Zeit-Features
    time_features = [
        'Tag', 'Monat', 'Jahr', 'Wochentag',
        'tag_sin', 'tag_cos', 'monat_sin', 'monat_cos',
        'Jahreszeit'
    ]
    
    # Alle Features kombinieren
    features = (weather_features + temp_features + 
               weather_cluster_features + holiday_features + time_features)
    
    print(f"Verwende {len(features)} Features: {features}")
    return features


In [ ]:
# 4. WARENGRUPPEN IDENTIFIZIEREN
# =============================================================================

def get_product_groups(df):
    """Identifiziert aktive Warengruppen in den Daten."""
    product_columns = [col for col in df.columns if col.startswith('Warengruppe_')]
    
    active_groups = []
    for col in product_columns:
        if df[col].sum() > 0:  # Nur Warengruppen mit Daten
            group_name = col.replace('Warengruppe_', '')
            active_groups.append((group_name, col))
    
    print(f"\nAktive Warengruppen: {[group[0] for group in active_groups]}")
    return active_groups


In [ ]:
# 5. MODELL TRAINING
# =============================================================================

def train_models(train_data, validation_data, features, product_groups):
    """Trainiert separate Modelle für jede Warengruppe."""
    print("\n" + "="*50)
    print("MODELL TRAINING")
    print("="*50)
    
    models = {}
    results = {}
    imputers = {}
    
    for group_name, group_col in product_groups:
        print(f"\nTrainiere Modell für {group_name}...")
        
        # Daten für diese Warengruppe filtern
        train_group = train_data[train_data[group_col] == 1].copy()
        val_group = validation_data[validation_data[group_col] == 1].copy()
        
        if len(train_group) == 0:
            print(f"  Keine Trainingsdaten für {group_name}")
            continue
            
        # Features und Target vorbereiten
        X_train = train_group[features]
        y_train = train_group['Umsatz']
        
        # Imputer für fehlende Werte
        imputer = SimpleImputer(strategy='median')
        X_train_imputed = imputer.fit_transform(X_train)
        
        # Modell trainieren
        model = LinearRegression()
        model.fit(X_train_imputed, y_train)
        
        # Validierung
        if len(val_group) > 0:
            X_val = val_group[features]
            X_val_imputed = imputer.transform(X_val)
            y_val = val_group['Umsatz']
            
            y_pred = model.predict(X_val_imputed)
            mae = mean_absolute_error(y_val, y_pred)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            
            results[group_name] = {
                'mae': mae,
                'rmse': rmse,
                'train_samples': len(train_group),
                'val_samples': len(val_group)
            }
            
            print(f"  Training: {len(train_group)} Datensätze")
            print(f"  Validation: {len(val_group)} Datensätze")
            print(f"  MAE: {mae:.2f}")
            print(f"  RMSE: {rmse:.2f}")
        
        models[group_name] = model
        imputers[group_name] = imputer
    
    return models, results, imputers

In [ ]:
# 6. VORHERSAGEN ERSTELLEN
# =============================================================================

def create_predictions(models, imputers, sample_sub, features):
    """Erstellt Vorhersagen für das Test-Set."""
    print("\n" + "="*50)
    print("VORHERSAGEN ERSTELLEN")
    print("="*50)
    
    # Warengruppen-Mapping
    group_mapping = {
        '1': 'Brot',
        '2': 'Brötchen', 
        '3': 'Croissant',
        '4': 'Konditorei',
        '5': 'Kuchen',
        '6': 'Saisonbrot'
    }
    
    predictions = []
    
    for idx, row in sample_sub.iterrows():
        id_str = str(row['id'])
        
        # ID parsen: YYMMDD + Warengruppe
        if len(id_str) == 7:
            date_part = id_str[:6]  # YYMMDD
            group_code = id_str[6]  # Warengruppe
            
            # Datum rekonstruieren
            year = 2000 + int(date_part[:2])
            month = int(date_part[2:4])
            day = int(date_part[4:6])
            
            try:
                date = datetime(year, month, day)
                
                # Features für Vorhersage erstellen
                feature_dict = {
                    'Tag': day,
                    'Monat': month,
                    'Jahr': year,
                    'Wochentag': date.weekday(),
                    # Zyklische Features
                    'tag_sin': np.sin(2 * np.pi * day/31),
                    'tag_cos': np.cos(2 * np.pi * day/31),
                    'monat_sin': np.sin(2 * np.pi * month/12),
                    'monat_cos': np.cos(2 * np.pi * month/12),
                    # Jahreszeit
                    'Jahreszeit': {12:0, 1:0, 2:0, 3:1, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3, 10:3, 11:3}[month],
                    # Dummy-Werte für Wetter und Feiertage (könnten verbessert werden)
                    'Bewoelkung': 5.0,
                    'Temperatur': 15.0,
                    'Windgeschwindigkeit': 10.0,
                    'Wettercode': 20.0,
                    'Wettercode_fehlt': 0,
                    'Temperatur_Delta': 0.0,  # Keine Änderung zum Vortag
                    # Wetter-Cluster (Standard: bewölkt)
                    'Wetter_Bewölkung': 1,
                    'Wetter_Sichtminderung': 0,
                    'Wetter_Nebel': 0,
                    'Wetter_Niederschlag': 0,
                    'Wetter_Schauer': 0,
                    'Wetter_Gewitter': 0,
                    'Wetter_Extremwetter': 0,
                    'Wetter_Fehlt': 0,
                    # One-Hot-Encoding für Temperatur (basierend auf 15°C = mild)
                    'Temp_kalt': 0,
                    'Temp_mild': 1,
                    'Temp_warm': 0,
                    'Temp_heiss': 0,
                    'ist_feiertag': 0,
                    'feiertag_vortag': 0,
                    'feiertag_folgetag': 0,
                    'KielerWoche': 0
                }
                
                # Vorhersage mit entsprechendem Modell
                group_name = group_mapping.get(group_code)
                if group_name and group_name in models:
                    X_pred = pd.DataFrame([feature_dict])[features]
                    X_pred_imputed = imputers[group_name].transform(X_pred)
                    prediction = models[group_name].predict(X_pred_imputed)[0]
                    prediction = max(0, prediction)  # Keine negativen Umsätze
                else:
                    prediction = 0  # Fallback
                
                predictions.append(prediction)
                
            except ValueError:
                predictions.append(0)  # Fallback bei ungültigem Datum
        else:
            predictions.append(0)  # Fallback bei ungültiger ID
    
    # Ergebnis-DataFrame erstellen
    result_df = sample_sub.copy()
    result_df['Umsatz'] = predictions
    
    print(f"Vorhersagen erstellt für {len(predictions)} IDs")
    print(f"Durchschnittlicher vorhergesagter Umsatz: {np.mean(predictions):.2f}")
    
    return result_df

In [ ]:
# 7. HAUPTFUNKTION
# =============================================================================

def main():
    """Hauptfunktion für die komplette Pipeline."""
    print("BÄCKEREI UMSATZ VORHERSAGE")
    print("="*50)
    
    # 1. Daten laden
    df, sample_sub = load_and_prepare_data()
    
    # 2. Daten aufteilen
    train_data, validation_data = split_data(df)
    
    # 3. Features definieren
    features = get_features(df)
    
    # 4. Warengruppen identifizieren
    product_groups = get_product_groups(df)
    
    # 5. Modelle trainieren
    models, results, imputers = train_models(train_data, validation_data, features, product_groups)
    
    # 6. Vorhersagen erstellen
    predictions_df = create_predictions(models, imputers, sample_sub, features)
    
    # 7. Ergebnisse speichern
    output_path = '/workspaces/bakery_sales_prediction/2_BaselineModel/predictions_linear_regression.csv'
    predictions_df.to_csv(output_path, index=False)
    
    print(f"\n" + "="*50)
    print("FERTIG!")
    print("="*50)
    print(f"Vorhersagen gespeichert in: {output_path}")
    
    # Zusammenfassung der Modell-Performance
    if results:
        print("\nModell-Performance (Validation):")
        for group, metrics in results.items():
            print(f"  {group}: MAE={metrics['mae']:.2f}, RMSE={metrics['rmse']:.2f}")

# =============================================================================
# AUSFÜHRUNG
# =============================================================================

if __name__ == "__main__":
    main()